In [ ]:
import pandas as pd
import polars as pl
import glob

In [ ]:
metadata = pl.read_csv(
    "../../Hoiho_Genomes_24Feb2024_JGG_3Pops.csv", separator="\t"
)
metadata

In [ ]:
files = glob.glob("../../random_snp_analysis_*.nex")

In [ ]:
# Reheader Nexus files
# Open the first file
for file in files:
    with open(file, "r") as f:
        print(file)

        # Remove the path and the ../../ from the file name
        file = file.split("/")[-1]
        # Output file name as "reheadered_" + file name
        output_file = "nexus_reheader/reheader_" + file
        # Also remove the colon from the file name
        with open(output_file, "w") as o:
            skip = False

            lines = f.readlines()
            # Skip the first 8 lines
            for i in range(8):
                o.write(lines[i])
            for line in lines[8:]:
                # Split by whitespace
                line_split = line.split("\t")
                if len(line_split) == 2:
                    id = line_split[0]
                    # See if we have a translation for the ID
                    sample_metadata = metadata.filter(pl.col("ID") == id)
                    if id in ["waitaha", "richdalei"]:
                        sample = f"{id}_{id}_{id}"
                    elif sample_metadata.is_empty():
                        # If not found, drop the _genomic at the end
                        sample = id.replace("_genomic", "")
                    else:
                        sample = sample_metadata.select("ID").to_pandas().values[0][0]
                        # Add Location and Population3 to the sample name
                        sample_metadata = sample_metadata.select("Location", "Population3")
                        sample_metadata = sample_metadata.to_pandas()
                        sample = sample + "_" + sample_metadata["Location"].values[0] + "_" + sample_metadata["Population3"].values
                        # Convert to str without the ndarray stuff
                        sample = str(sample[0])
                        
                    o.write(f"{sample} {"\t".join(line_split[1:])}\n")
                else:
                    o.write(line)

In [ ]:
# Glob all files starting with out-ptg*.fasta
files = glob.glob("out-ptg*.fasta")

 So the accessions should have the following format:

UniqueIdentifier_samplelocation_population

and ensure that "uniqueidentifier" does not have any _ inside it

So for example A1 would be renamed to:

A1_OP_Northern

while Eudyptes_robustus can become

Eudyptes.robustus_robustus_robustus

In [ ]:
# Open the first file
for file in files:
    with open(file, "r") as f:

        # Output file name as "reheadered_" + file name
        output_file = "reheader_" + file
        # Also remove the colon from the file name
        output_file = output_file.replace(":", "_")
        with open(output_file, "w") as o:
            skip = False

            lines = f.readlines()
            for line in lines:
                if line.startswith(">"):
                    # Get the sample name (drop the ">")
                    sample = line[1:].strip()
                    if sample == "a9":
                        skip = True
                        continue
                    else: 
                        skip = False
                    
                    # Get the metadata for the sample
                    sample_metadata = metadata.filter(pl.col("ID") == sample)
                    # If not found, change to sample name 
                    # while Eudyptes_robustus can become 
                    # Eudyptes.robustus_robustus_robustus

                    if sample_metadata.shape[0] == 0:
                        sample = sample.replace("_", ".")
                        # Repeat what's after the period twice such as Eudyptes.robustus_robustus_robustus
                        sample = sample + "_" + sample.split(".")[1] + "_" + sample.split(".")[1]
                    else:
                        sample = sample_metadata.select("ID").to_pandas().values[0][0]
                        # Add Location and Population3 to the sample name
                        sample_metadata = sample_metadata.select("Location", "Population3")
                        sample_metadata = sample_metadata.to_pandas()
                        sample = sample + "_" + sample_metadata["Location"].values[0] + "_" + sample_metadata["Population3"].values
                        # Convert to str without the ndarray stuff
                        sample = str(sample[0])
                        
                    print(sample)
                    o.write(">" + sample + "\n")
                else:
                    if skip:
                        continue
                    o.write(line)


            
